1

`surface1.semi_aperture` is computed from paraxial rays as |ya| + |yb|, coming from paraxial marginal/chief rays returned by `optic.paraxial.marginal_ray()` and `optic.paraxial.chief_ray()`

`semi_aperture` is initialized to None and only set when the paraxial update runs. Call `optic.update_paraxial` to ensure it's not `None`. 

`optic.update()` only calls `update_paraxial()` if certain surface types exist (e.g., chebyshev, zernike, forbes). Otherwise, it skips it

2

`surface1.y[-1]` is just the last ray’s y‑coordinate recorded during ray tracing. The `Surface.trace()` method records ray data via _record, which copies the y‑values from the actual traced rays in their current ordering (not necessarily sorted or maximum). Thus, `y[-1]` is only “the last ray in the array,” not “the marginal ray” or “the max ray.”

Ray clipping can also change `surface1.y`, because apertures can clip rays before recording happens. That means `surface1.y` only reflects rays that survive the aperture filter, further decoupling it from the paraxial semi‑aperture calculation.

`surface.y` is reset to an empty array on every surface reset, and it is only filled after a real ray trace.

`Surface._record()` only populates `self.y` after `Surface.trace()` is called (real rays)

3

`rays.i` corresponds to the final ray state, which is effectively the last surface / image surface result after all effects are applied

Each surface records the ray intensity that hits it during tracing:

`self.intensity = be.copy(be.atleast_1d(rays.`

This happens in `Surface._record()` after every surface trace.

`surface.intensity` → intensity of rays at that surface

`surface_group.intensity` → stacked intensities for all surfaces with data (via `SurfaceGroup.intensity`).

4.

`optic.draw(num_rays=10)` traces rays internally. `draw()` does not use the rays you traced earlier. It runs its own ray trace via the visualization pipeline, using its own distribution and number of rays.

`Optic.draw()` → `OpticViewer.view()` → `Rays2D.plot()` → `Rays2D._trace()` → `optic.trace(...)`
This means `draw()` calls a fresh trace and overwrites `surface_group` data.

5.

`Optic.trace()` uses `RealRayTracer.trace()` to generate rays and trace them, and returns a `RealRays` object whose i is the intensity vector for those traced rays.